In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

KeyboardInterrupt: 

In [2]:
# Define transforms
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
# Define the dataset and dataloader
train_dataset = datasets.ImageFolder('/Users/shaamerk/Desktop/class/Squishy Robotics/gunshot/noisedetection/train/', transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.ImageFolder('/Users/shaamerk/Desktop/class/Squishy Robotics/gunshot/noisedetection/test/', transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the model

model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(64, 192, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(192, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    nn.Linear(256 * 6 * 6, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 2)
)


In [7]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [6]:
# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

KeyboardInterrupt: 

USING RESNET18 BELOW

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models

# Load a pre-trained ResNet-34 model and replace the final fully-connected layer
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch: %d, Loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

Epoch: 1, Loss: 0.380
Epoch: 2, Loss: 0.278
Epoch: 3, Loss: 0.283
Epoch: 4, Loss: 0.237
Epoch: 5, Loss: 0.230
Epoch: 6, Loss: 0.373
Epoch: 7, Loss: 0.331
Epoch: 8, Loss: 0.348
Epoch: 9, Loss: 0.300
Epoch: 10, Loss: 0.316


TEST CELL for RESNET18

In [9]:
device = torch.device('cpu')

# Put the model in evaluation mode
model.eval()

# Initialize variables to keep track of loss and accuracy
test_loss = 0
accuracy = 0

# Loop over the test data in batches
for images, labels in test_loader:
    # Move the data to the device
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    output = model(images)

    # Calculate the loss
    loss = criterion(output, labels)

    # Update the test loss
    test_loss += loss.item()*images.size(0)

    # Calculate the accuracy
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(labels.data.view_as(pred))
    accuracy += torch.mean(correct_tensor.type(torch.FloatTensor)).item()*images.size(0)

# Calculate the average loss and accuracy
test_loss = test_loss/len(test_loader.dataset)
accuracy = accuracy/len(test_loader.dataset)

print('Test Loss: {:.6f}\n'.format(test_loss))
print('Test Accuracy: {:.2f}%\n'.format(accuracy*100))

Test Loss: 0.264635

Test Accuracy: 85.13%



RESNET 34

In [11]:
# Load a pre-trained ResNet-34 model and replace the final fully-connected layer
model2 = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
num_ftrs = model2.fc.in_features
model2.fc = nn.Linear(num_ftrs, 2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch: %d, Loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

Epoch: 1, Loss: 0.499
Epoch: 2, Loss: 0.417
Epoch: 3, Loss: 0.318
Epoch: 4, Loss: 0.349
Epoch: 5, Loss: 0.390
Epoch: 6, Loss: 0.336
Epoch: 7, Loss: 0.316
Epoch: 8, Loss: 0.297
Epoch: 9, Loss: 0.259
Epoch: 10, Loss: 0.267


Testing for Resnet34 10 Epochs

In [12]:
device = torch.device('cpu')

# Put the model in evaluation mode
model2.eval()

# Initialize variables to keep track of loss and accuracy
test_loss = 0
accuracy = 0

# Loop over the test data in batches
for images, labels in test_loader:
    # Move the data to the device
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    output = model2(images)

    # Calculate the loss
    loss = criterion(output, labels)

    # Update the test loss
    test_loss += loss.item()*images.size(0)

    # Calculate the accuracy
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(labels.data.view_as(pred))
    accuracy += torch.mean(correct_tensor.type(torch.FloatTensor)).item()*images.size(0)

# Calculate the average loss and accuracy
test_loss = test_loss/len(test_loader.dataset)
accuracy = accuracy/len(test_loader.dataset)

print('Test Loss: {:.6f}\n'.format(test_loss))
print('Test Accuracy: {:.2f}%\n'.format(accuracy*100))

Test Loss: 8.227090

Test Accuracy: 48.32%



Resnet 34 with 3 epochs: Accuracy is 65%
Resnet 34 with 5 epochs: Accuracy is 

In [19]:
# Load a pre-trained ResNet-34 model and replace the final fully-connected layer
model3 = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
num_ftrs = model3.fc.in_features
model3.fc = nn.Linear(num_ftrs, 2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(), lr=0.01)

# Train the model
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model3(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch: %d, Loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

Epoch: 1, Loss: 0.826
Epoch: 2, Loss: 0.602
Epoch: 3, Loss: 0.542
Epoch: 4, Loss: 0.539
Epoch: 5, Loss: 0.466


In [20]:
device = torch.device('cpu')

# Put the model in evaluation mode
model3.eval()

# Initialize variables to keep track of loss and accuracy
test_loss = 0
accuracy = 0

# Loop over the test data in batches
for images, labels in test_loader:
    # Move the data to the device
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    output = model3(images)

    # Calculate the loss
    loss = criterion(output, labels)

    # Update the test loss
    test_loss += loss.item()*images.size(0)

    # Calculate the accuracy
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(labels.data.view_as(pred))
    accuracy += torch.mean(correct_tensor.type(torch.FloatTensor)).item()*images.size(0)

# Calculate the average loss and accuracy
test_loss = test_loss/len(test_loader.dataset)
accuracy = accuracy/len(test_loader.dataset)

print('Test Loss: {:.6f}\n'.format(test_loss))
print('Test Accuracy: {:.2f}%\n'.format(accuracy*100))

Test Loss: 0.745224

Test Accuracy: 65.62%

